# 訓練機器學習進行分類

## 目錄

- 人工神經網路
- 利用 Iris 資料集訓練感知器

# 在 Python 中實作感知器學習演算法

## 實作感知器 

In [ ]:
import numpy as np


class Perceptron:
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        # 學習率 Learning rate (介於 0.0 到 1.0 之間) 
        self._eta = eta
        # 回合數 Epoch
        self._n_iter = n_iter
        # 用於隨機權重初始化的隨機數產生器種子。
        self._random_state = random_state #
        # 權重
        self._weights = np.asarray([]) # 1d-array
        # 損失
        self.errors = [] # list

    # 擬合模型
    def fit(self, x, y):
        rgen = np.random.RandomState(self._random_state)
        self._weights = rgen.normal(loc=0.0, scale=0.01, size=1 + x.shape[1])
        self.errors = []

        for _ in range(self._n_iter):
            errors = 0
            for xi, target in zip(x, y):
                update = self._eta * (target - self.predict(xi))
                self._weights[1:] += update * xi
                self._weights[0] += update
                errors += int(update != 0.0)
            self.errors.append(errors)
        return self

    # 計算網路輸入
    def net_input(self, x):
        return np.dot(x, self._weights[1:]) + self._weights[0]

    # 預測類別標記
    def predict(self, x):
        return np.where(self.net_input(x) >= 0.0, 1, -1)

## 利用 Iris 資料集來訓練感知器(Perceptron)

使用網路中的開放資料集，來訓練感知器。

### 讀取 Iris 資料集 (網路)

In [ ]:
import pandas as pd

iris_dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
df = pd.read_csv(iris_dataset_url,
                 header=None,
                 names=["花萼長度", "花萼寬度", "花瓣長度", "花瓣寬度", "標籤"],
                 encoding='utf-8')


df.tail()

### 讀取 Iris 的資料集 (本地)

In [ ]:
df = pd.read_csv("./resources/iris_data.csv", 
                 header=None,
                 names=["花萼長度", "花萼寬度", "花瓣長度", "花瓣寬度", "標籤"], 
                 encoding='utf-8')
df.tail()

讓 MatPlotLib 可以使用中文字型

In [ ]:
import matplotlib as mlp
from matplotlib.font_manager import fontManager

fontManager.addfont('./resources/NotoSansTC.ttf')
mlp.rc("font", family="Noto Sans TC")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# 選擇 ‘setosa’ 與 ‘versicolor’
y_label = df.iloc[0:100, 4].values
y_label = np.where(y_label == 'Iris-setosa', -1, 1)

# 提取花萼長度與花瓣長度
x_train = df.iloc[0:100, [0, 2]].values

# 資料繪圖
plt.scatter(x_train[:50, 0], x_train[:50, 1],
            color='red', marker='o', label='setosa (山鳶尾)')
plt.scatter(x_train[50:100, 0], x_train[50:100, 1],
            color='blue', marker='x', label='versicolor (變色鳶尾)')

plt.xlabel('花萼長度 [cm]')
plt.ylabel('花瓣長度 [cm]')
plt.legend(loc='upper left')

plt.show()

In [ ]:
ppn = Perceptron(eta=0.1, n_iter=10)

ppn.fit(x_train, y_label)

plt.plot(range(1, len(ppn.errors) + 1), ppn.errors, marker='o')
plt.xlabel('回合')
plt.ylabel('錯誤率')

plt.show()

In [ ]:
from matplotlib.colors import ListedColormap


def plot_decision_regions(x, y, classifier, resolution=0.02):
    # 設定標記和顏色
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # 繪製決策邊界
    x1_min, x1_max = x[:, 0].min() - 1, x[:, 0].max() + 1
    x2_min, x2_max = x[:, 1].min() - 1, x[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    z = z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    
    # 繪製訓練資料點
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=x[y == cl, 0],
                    y=x[y == cl, 1],
                    alpha=0.8,
                    c=colors[idx],
                    marker=markers[idx],
                    label=cl)

In [ ]:
plot_decision_regions(x_train, y_label, classifier=ppn)

plt.xlabel('花萼長度 [cm]')
plt.ylabel('花瓣長度 [cm]')
plt.legend(loc='upper left')

plt.show()

In [ ]:
class AdalineGD(object):
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self._eta = eta
        self._n_iter = n_iter
        self._random_state = random_state
        self._weights = np.array([])
        self.cost = []

    def fit(self, x, y):
        rgen = np.random.RandomState(self._random_state)
        self._weights = rgen.normal(loc=0.0, scale=0.01, size=1 + x.shape[1])
        self.cost = []

        for i in range(self._n_iter):
            net_input = self.net_input(x)
            output = self.activation(net_input)
            errors = (y - output)
            # w = w + \Delta{w} => w += \Delta{w} => w += \eta(y-\hat{y})x^T
            self._weights[1:] += self._eta * x.T.dot(errors)
            self._weights[0] += self._eta * errors.sum()
            cost = (errors**2).sum() / 2.0 # 誤差平方和 (Sum of the Squared Errors, SSE)
            self.cost.append(cost)
        return self

    def net_input(self, x):
        # z = xw + b
        return np.dot(x, self._weights[1:]) + self._weights[0]

    def activation(self, x):
        return x

    def predict(self, x):
        return np.where(self.activation(self.net_input(x)) >= 0.0, 1, -1)

In [ ]:
# standardize features
X_std = np.copy(x_train)
X_std[:, 0] = (x_train[:, 0] - x_train[:, 0].mean()) / x_train[:, 0].std()
X_std[:, 1] = (x_train[:, 1] - x_train[:, 1].mean()) / x_train[:, 1].std()

In [ ]:
ada_gd = AdalineGD(n_iter=15, eta=0.01)
ada_gd.fit(X_std, y_label)

plot_decision_regions(X_std, y_label, classifier=ada_gd)
plt.title('自適應線性神經元 - 梯度下降')
plt.xlabel('花萼長度 [標準化]')
plt.ylabel('花瓣長度 [標準化]')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

plt.plot(range(1, len(ada_gd.cost) + 1), ada_gd.cost, marker='o')
plt.xlabel('回合')
plt.ylabel('誤差平方和')

plt.tight_layout()
plt.show()